In [ ]:
from collections import defaultdict, deque
import numpy as np
import math
import sys
import gc
from copy import deepcopy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
% cd /content/drive/MyDrive/

In [ ]:
# del doc_word_mapper
gc.collect() # in case if ram keeps increasing

88

In [ ]:
label_count_mapper = np.zeros((20, 20)) # since i know total number of docs I'm making this array directly
class_docs = {} # how much docs each class has
train_y = [] # training labels (1-20)
conf_matrix = np.zeros((20, 20))

with open('./news_data/train.label', 'r') as fp:
  for line in fp:
    label = int(line.strip())
    # its good to point out that doc_id 'n' will have label at 'n-1'th place
    # in simple words doc0 will have label at train_y[0] which will be between 1-20
    train_y.append(label)
    class_docs[label] = class_docs.get(label, 0) + 1
    
previous_doc = 1
freq = {} # (word, class): count
with open('./news_data/train.data', 'r') as fp:
  word_ids = [] # this will basically contain the words and their counts
  for line in fp:
    doc_id, word_id, count = map(int, line.strip().split(' '))
    # this means we have got all the words of our doc now we can take care of frequencies
    # for doc1
    # (1,5), (2,10), ...
    # doc1 => class1
    if previous_doc != doc_id:
      # doc N will have label at (N-1)th place
      class_of_doc = train_y[previous_doc-1] # getting class of doc
      for word, word_count in word_ids:
        # we are gonna have count of each word in each class in this dictionary
        # example. hello, class1 => 20 times
        freq[(word, class_of_doc)] = freq.get((word, class_of_doc), 0) + word_count
      previous_doc = doc_id
      word_ids = [(word_id, count)]
    else:
      word_ids.append((word_id, count)) # appending word_id
# for the last doc
class_of_doc = train_y[previous_doc-1] # getting class of doc
for word, count in word_ids:
  freq[(word, class_of_doc)] = freq.get((word, class_of_doc), 0) + word_count
del word_ids

vocab = set([pair[0] for pair in freq.keys()])
v_len = len(vocab)

# finding how much words each class has
class_words = {}
for pair, word_count in freq.items():
  class_ = pair[1]
  class_words[class_] = class_words.get(class_, 0) + word_count

total_docs = len(train_y) # this should be 11269 if using their indexing (see train.data)
# sum(class_docs.values()) # again, this should be same as total_docs

prob_class = {}
prob_word_class = {}

# finding probability of each class
for i in class_docs:
  prob_class[i] = class_docs[i]/total_docs

# findinf probability of each word in each class
# we are doing this with smoothing too, for better results
for word in vocab:
  for class_ in class_words:
    freq_class = freq.get((word, class_), 0)
    # word/class
    prob_word_class[(word, class_)] = (freq_class + 1)/(class_words[class_] + v_len)

In [ ]:

"""
so now 
we have prior probability of each class => prob_class
we have total count of each class => label_count_mapper

doc: word1 word2 ...
prob[class/givenwords] = prob[class] * prob[word1/class] * prob[word2/class]....

and find max probability to assign the class

Problem: because docs are big we have hell of words
so our probabilities became very very very small(which computer considers as 0)
so we will consider taking log on both sides

log(prob[class]) + log(prob[word1/class]) + log(prob[word2/class])....

moreover we also have occurencies of each word.
so
prob = prob[class] * (prob[word1/class] * prob[word1/class]... )N times....
so
prob = prob[class] * prob[word1/class]^occurence_word1 * prob[word2/class]^occurence_word2

hence
log(prob) = log(prob[class]) + (occurence_word1 * log(word1/class)) + ...
we do this for all 20 classes
and see which has biggest value

"""

In [ ]:
conf_matrix = np.zeros((20,20))
previous_doc = 1
y_expected = []
y_actual = []
with open('./news_data/test.label', 'r') as fp:
  for line in fp:
    y_expected.append(int(line.strip()))
total_test_docs = len(y_expected)
correct_classified = 0
with open('./news_data/test.data', 'r') as fp:
  word_ids = [] # this will basically contain the words and their counts
  j = 0
  for line in fp:
    doc_id, word_id, count = map(int, line.strip().split(' '))
    if previous_doc != doc_id:
      probs = deepcopy(prob_class)
      for i in probs:
        probs[i] = np.log(probs[i])
      for word, word_count in word_ids:
        for class_ in range(1,21):
          # print(prob_word_class.get((word, class_), 1e-5), end=' ')
          probs[class_] = probs[class_]  +  word_count * np.log(prob_word_class.get((word, class_), 1e-5))
          # probs[i] = probs[i] + word_count * p_word_class.get((word, i), 1e-4)
      _max_class = 1
      _max_val = - np.inf
      for i in probs:
        if probs[i] > _max_val:
          _max_val = probs[i]
          _max_class = i
      y_actual.append(_max_class)
      # print(_max_class, end = ' ')
      if y_expected[j] == _max_class:
        correct_classified+=1
      conf_matrix[_max_class-1][y_expected[j]-1] = conf_matrix[_max_class-1][y_expected[j]-1] + 1
      j += 1
      previous_doc = doc_id
      word_ids = [(word_id, count)]
    else:
      word_ids.append((word_id, count)) # appending word_id

# this is for the last word_ids (code can be taken into function to remove redunduncy)
for word, word_count in word_ids:
  for class_ in range(1,21):
    # print(prob_word_class.get((word, class_), 1e-5), end=' ')
    probs[class_] = probs[class_]  +  word_count * np.log(prob_word_class.get((word, class_), 1e-5))
    # probs[i] = probs[i] + word_count * p_word_class.get((word, i), 1e-4)
_max_class = 1
_max_val = - np.inf
for i in probs:
  if probs[i] > _max_val:
    _max_val = probs[i]
    _max_class = i
# print(_max_class, end = ' ')
y_actual.append(_max_class)
if y_expected[j] == _max_class:
  correct_classified+=1
conf_matrix[_max_class-1][y_expected[j]-1] = conf_matrix[_max_class-1][y_expected[j]-1] + 1

incorrect_classified = total_test_docs - correct_classified
print(correct_classified/total_test_docs)

0.7846768820786143


In [ ]:
# # FOR F1 score
from sklearn.metrics import f1_score
f1_score(y_expected, y_actual, average=None)

array([0.72617247, 0.71411339, 0.64806202, 0.67709497, 0.7537415 ,
       0.79947575, 0.734375  , 0.8436019 , 0.9121447 , 0.91578947,
       0.95037221, 0.82045455, 0.71625344, 0.85375494, 0.86709677,
       0.79704017, 0.77234803, 0.87601078, 0.58730159, 0.50969529])

In [ ]:
conf_matrix

array([[240.,   3.,   3.,   0.,   1.,   0.,   0.,   0.,   0.,   4.,   2.,
          0.,   2.,  10.,   4.,   7.,   1.,  12.,   7.,  47.],
       [  0., 296.,  33.,   8.,   8.,  42.,   9.,   1.,   1.,   0.,   0.,
          4.,  17.,   7.,   8.,   2.,   0.,   1.,   1.,   2.],
       [  0.,   6., 209.,  15.,   9.,   8.,   4.,   0.,   0.,   0.,   0.,
          1.,   0.,   1.,   0.,   1.,   0.,   0.,   0.,   0.],
       [  0.,  12.,  60., 303.,  36.,  11.,  46.,   2.,   0.,   1.,   0.,
          1.,  28.,   3.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   8.,  10.,  22., 277.,   2.,  21.,   0.,   0.,   1.,   0.,
          2.,   7.,   0.,   0.,   1.,   1.,   0.,   0.,   0.],
       [  1.,  21.,  30.,   2.,   2., 305.,   0.,   1.,   0.,   3.,   0.,
          1.,   3.,   0.,   1.,   2.,   0.,   0.,   1.,   0.],
       [  0.,   1.,   0.,   5.,   5.,   1., 235.,   5.,   1.,   2.,   0.,
          1.,   1.,   0.,   0.,   0.,   1.,   0.,   0.,   0.],
       [  0.,   3.,   1.,   6.,   4.,   0

In [ ]:
# ROCHIO Algos
# each doc will be of N space where N is vocab length(set of unique words allover)

# first we find centroid of each class
# so say to find centroid of class 1, we find average N dimensional vector of documents of class 1 in training
# we can say this is end of training phase

# now take a doc from test phase and find its cosin similarity with each class centroid
# and we give it label of one which is of closest angle

In [ ]:
text = """Let me take you down
'Cause I'm going to Strawberry Fields
Nothing is real
And nothing to get hung about
Strawberry Fields forever
Living is easy with eyes closed
Misunderstanding all you see
It's getting hard to be someone
But it all works out
It doesn't matter much to me
Let me take you down
'Cause I'm going to Strawberry Fields
Nothing is real
And nothing to get hung about
Strawberry Fields forever
No one I think is in my tree
I mean it must be high or low
That is you can't, you know, tune in
But it's all right
That is I think it's not too bad
Let me take you down
'Cause I'm going to Strawberry Fields
Nothing is real
And nothing to get hung about
Strawberry Fields forever
Always, no sometimes, think it's me
But you know I know when it's a dream
I think I know I mean a yes
But it's all wrong
That is I think I disagree
Let me take you down
'Cause I'm going to Strawberry Fields
Nothing is real
And nothing to get hung about
Strawberry Fields forever
Strawberry Fields forever
Strawberry Fields forever"""

def get_index(char: str) -> int:
    if char.isalpha():
      return ord(char.lower()) - 97 # so a comes on 0, b-1, ...z-25
    else:
      return 26

def get_occurence_matrix(text):
  occurence_matrix = np.zeros((27,27))
  l = len(text)
  # a => 97, z => 122
  for i in range(1, l):
    current = text[i-1]
    next = text[i]
    current_index = get_index(current)
    next_index = get_index(next)
    occurence_matrix[current_index, next_index] += 1
  return occurence_matrix

26

In [ ]:
occurence_matrix = get_occurence_matrix(text)
def get_probability_matrix(occurence_matrix):
  prob_matrix =  occurence_matrix/occurence_matrix.sum(axis=1, keepdims=True)
  prob_matrix[np.isnan(prob_matrix)] = 0 # removing NaNs
  return prob_matrix
get_probability_matrix(occurence_matrix)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


array([[0.        , 0.07843137, 0.        , 0.01960784, 0.        ,
        0.        , 0.01960784, 0.        , 0.        , 0.        ,
        0.07843137, 0.17647059, 0.01960784, 0.15686275, 0.        ,
        0.        , 0.        , 0.01960784, 0.01960784, 0.07843137,
        0.07843137, 0.        , 0.19607843, 0.        , 0.01960784,
        0.        , 0.03921569],
       [0.04761905, 0.        , 0.        , 0.        , 0.57142857,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.19047619,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.19047619, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.71428571, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.14285714, 0.        , 0.        ,
        0.        , 0.14285714, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.    